In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser2"
password = "CS340"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
df = df.iloc[:,1:] # Initializes the df variable


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div(className='row', style={'display' : 'flex', 'height': '96vh'},children=[
    #Side bar for tools and interactibles
    html.Div(style={'width': '60vh'},children=[
    html.Center([
            html.Div(
                html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                     style={'width': '150px', 'height': '150px'}))
            ),
            html.Div(
                html.Center(html.B(html.H2('Dan Stulls')))
            ),     
            html.Div(
                html.Center(html.B(html.H2('SNHU-340')))
            ),     
            html.Div(
                html.Center(html.B(html.H2('Dashboard')))
            ),
            ]),   
        
    html.Center(html.H5(children='By Daniel Stull 04/15/2023')),
    html.Hr(),
    
    #input methods and unique identifier
        html.Div(
                html.B(html.H5('Existing Accounts:'))
            ),
    dcc.Input(
            id="input_user".format("text"),
            type="text",
            placeholder="Username"),
        dcc.Input(
            id="input_passwd".format("password"),
            type="password",
            placeholder="Password"),
        html.Button('Sign In', id='submit-val', n_clicks=0),
        
    html.Hr(),
    html.Div(
        #Radio Items to select the rescue filter options
        dcc.Dropdown(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MntWild'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DsTr'},
            {'label': 'No Filter', 'value': 'RESET'}
            ],
            value='RESET',
            style={'width': '100%'}
        )
    ),
    
]),
    
#Separating bar from tools and output  
html.Hr(),
    
#Main Output plane
html.Div(style ={"overflow": "scroll"}, children=[
    
    #datatable parameters
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        column_selectable=False,
        editable=False,
        filter_action="native",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        page_current=0,
        page_size=15,

    ),
    html.Br(),
    html.Hr(),
    
    #Places graph and map
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
    html.Div(
        id='graph-id',
        className='col s12 m6'),
    html.Div(id='map-id',className='col s12 m6'),   
             ])
        
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        if filter_type == 'Water':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesa Bay Retr Mix'},
                                                              {'breed': 'Newfoundland Mix'},
                                                              {'breed': 'Newfoundland/Labrador Retriever'},
                                                              {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                              {'breed': 'Newfoundland/Great Pyrenees'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        #adjusts the read request for the desired dog type and status
        elif filter_type == 'MntWild':
            # breeds and ages 
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Siberian Husky'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        #adjusts the read request for the desired dog type and status
        elif filter_type == 'DsRt':
            #breeds and ages 
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                })))
        
        #resets the search to nothing to allow all results to be displayed
        elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # imports data from interactive table
    dff = pd.DataFrame.from_dict(viewData)
    #graph variables
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
    # imports the currently viewed data
    dff = pd.DataFrame.from_dict(viewData)
    #determines the default status
    if selected_rows == []:
        selected_rows = [0]
    # creates a map for a single selected row or the default
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ])
            ])
        ]
    #creates a map with 2 markers
    elif len(selected_rows) == 2:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ])
            ])
        ]
    #creates a map with 3 markers
    elif len(selected_rows) == 3:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ])
            ])
        ]
    #creats a map with 4 markers
    elif len(selected_rows) == 4:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[3],13]), (dff.iloc[selected_rows[3],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[3],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[3],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[3],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[3],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[3],15])
                    ])
                ])
            ])
        ]
    elif len(selected_rows) >= 5:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[3],13]), (dff.iloc[selected_rows[3],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[3],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[3],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[3],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[3],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[3],15])
                    ])
                ]),
                dl.Marker(position=[(dff.iloc[selected_rows[4],13]), (dff.iloc[selected_rows[4],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[4],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[4],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[4],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[4],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[4],15])
                    ])
                ])
            ])
        ]
    


app